In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import keras as k

Using TensorFlow backend.


In [3]:
dimer_atoms = 4
solvent_atoms = 72
numberOfLayers = 1
total_atoms = dimer_atoms + solvent_atoms
neurons = 3
dim = 2

In [4]:
x = tf.placeholder(tf.float32, [None, dimer_atoms + solvent_atoms - 1], name = 'input' )
y = tf.placeholder(tf.float32, [None, 2* solvent_atoms], name='output')
w = tf.Variable(tf.random.truncated_normal([total_atoms - 1],mean=0, stddev= 2 ), name='weights1' ) 
'''
for i in range(solvent_atoms):
    w.append ( tf.Variable(tf.random.truncated_normal([total_atoms - 1],mean=0, stddev=1 / np.sqrt(n_atoms)), name='weights1' ) )
w= tf.Variable(w)
'''
hBeforeActivation = tf.multiply( w , x[:-1] ) #element wise multiplication
h = tf.nn.relu( tf.cumsum(hBeforeActivation) ) #cumulativeSum


In [5]:
v = tf.Variable(tf.random.truncated_normal([total_atoms -1, 2* solvent_atoms ], mean=0, stddev=2))
y = tf.linalg.matvec(tf.transpose(v), h )

In [6]:
def loss(yPred, output):
    normal = tfp.trainable_distributions.normal(output[0])
    loss = -tf.reduce_mean(normal.log_prob(xInput))
    train_op = tf.train.AdamOptimizer(learning_rate=2.**-5).minimize(loss)
    mse = tf.reduce_mean(tf.squared_difference(y, normal.mean()))


In [7]:
batch_size = 128
epochs = 100

In [8]:
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy',
              metrics=['myloss'])

NameError: name 'model' is not defined

In [153]:
import torch
import torch.nn as nn
from pathlib import Path
import torchvision.transforms as transforms
import torch.optim as optim


In [253]:
#RNADE
class RNADE(nn.Module):
    def __init__(self,dimer_atoms,solvent_atoms):
        super(RNADE, self).__init__()
        self.dimer_atoms = dimer_atoms
        self.solvent_atoms = solvent_atoms
        self.total_dims = self.dimer_atoms + self.solvent_atoms
        self.i2h = nn.Linear(self.total_dims-1, self.total_dims-1)
        self.h2o = nn.Linear(self.total_dims-1, self.total_dims-self.dimer_atoms)
        self.D = self.total_dims
        self.H = 128
        self.params = nn.ParameterDict({
            "V" : nn.Parameter(torch.randn(self.D, self.H)),
            "b" : nn.Parameter(torch.zeros(self.D)),
            "V2" : nn.Parameter(torch.randn(self.D, self.H)),
            "b2" : nn.Parameter(torch.zeros(self.D)),
            "W" : nn.Parameter(torch.randn(self.H, self.D)),
            "c" : nn.Parameter(torch.zeros(1, self.H)),
        })
        nn.init.xavier_normal_(self.params["V"])
        nn.init.xavier_normal_(self.params["V2"])
        nn.init.xavier_normal_(self.params["W"])
        
    def forward(self, x):
        ai = self.params["c"].expand(x.size(0), -1)   #B x H
        print(ai.size())
        final_prob = []
        a=[]
        m = []
        for d in range(self.D):
            h_i = torch.relu(ai) #B x H
            #print(h_i.size())
            alphai = torch.sigmoid( h_i.mm(self.params["V"][d:d+1,:].t() ) + self.params["b"][d:d+1] )*6 + pow(10,-20) #  BxH *  Hx1  
            meani = h_i.mm(self.params["V"][d:d+1,:].t() ) + self.params["b2"][d:d+1]
            a.append(alphai)
            m.append(meani)
            final_prob.append(torch.cat((meani,alphai),1) ) 
            #print(final_prob[-1].size())
            ai = x[:, d:d+1].mm(self.params["W"][:, d:d+1].t()) + ai #Bx1 * 1xH =  BxH
        #print(final_prob)
        #final_prob = torch.cat(final_prob,-1)
        #print(a[-1], m[-1])
        a = torch.cat(a,1)
        m = torch.cat(m,1)
        final_prob = torch.stack([m,a])
       
        return final_prob

In [254]:
from math import *

In [255]:
def lossFunct(output,pred):
    print(output.size())
    alpha = output[1, :,4:]
    mean = output[0, :, 4:]
    print(mean.size())
    loss = torch.exp( -0.5 *  ( (pred[:,4:]- mean)/alpha)**2  )/alpha
    print("Loss:")
    print(loss)
    print(-torch.log( loss ).sum(axis= 1).sum())
    return -torch.log( loss ).sum(axis= 1).sum()

In [256]:
print(device)

cpu


In [257]:
model = RNADE(4,72).to(device)
loss = lossFunct

In [258]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [259]:
model.train()

RNADE(
  (i2h): Linear(in_features=75, out_features=75, bias=True)
  (h2o): Linear(in_features=75, out_features=72, bias=True)
  (params): ParameterDict(
      (V): Parameter containing: [torch.FloatTensor of size 76x128]
      (V2): Parameter containing: [torch.FloatTensor of size 76x128]
      (W): Parameter containing: [torch.FloatTensor of size 128x76]
      (b): Parameter containing: [torch.FloatTensor of size 76]
      (b2): Parameter containing: [torch.FloatTensor of size 76]
      (c): Parameter containing: [torch.FloatTensor of size 1x128]
  )
)

In [260]:

for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        print(data.size())
        optimizer.zero_grad()
        x_hat = model(data)
        print(len(x_hat))
        loss = lossFunct(x_hat, data)
        loss.backward()
        optimizer.step()
        print(loss.item())
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[0.2107, 0.1904, 0.1849,  ..., 0.2807, 0.1619, 0.2091],
        [0.2258, 0.1855, 0.2002,  ..., 0.2604, 0.1417, 0.1925],
        [0.2066, 0.1807, 0.1992,  ..., 0.1956, 0.1871, 0.1763],
        ...,
        [0.1967, 0.1853, 0.1920,  ..., 0.3296, 0.1456, 0.2518],
        [0.2054, 0.2039, 0.1919,  ..., 0.2440, 0.1936, 0.1807],
        [0.2175, 0.1874, 0.1910,  ..., 0.2644, 0.1513, 0.2030]],
       grad_fn=<DivBackward0>)
tensor(29952.7051, grad_fn=<NegBackward>)
29952.705078125
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[0.1996, 0.1863, 2.3538,  ..., 0.0000, 0.0000, 0.0000],
        [0.2086, 0.1901, 2.1705,  ..., 0.0000, 0.0000, 0.0000],
        [0.2038, 0.1865, 2.6167,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.2007, 0.1875, 2.2152,  ..., 0.0000, 0.0000, 0.0000],
        [0.2218, 0.1839, 1.9731,  ..., 0.000

nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 7

nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 7

nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 7

nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 7

...

nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 7

2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan,

2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan,

2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan,

2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<NegBackward>)
nan
torch.Size([256, 76])
torch.Size([256, 128])
2
torch.Size([2, 256, 76])
torch.Size([256, 72])
Loss:
tensor([[nan, nan, nan,  ..., nan, nan,

In [261]:
a = torch.tensor([[ [0, 2], [0,1], [0,0.5] ], [ [0, 2], [0,1], [0,0.5] ] ] )

In [262]:
output = torch.tensor( [ [1, 2,3], [3,1, 2]] )

In [263]:
lossFunct(a,output)

torch.Size([2, 3, 2])
torch.Size([3, 0])


RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 0

In [ ]:
torch.tensor([[1,2], [2,3],[3,4]]).t()

In [12]:
trajdict = np.load('output.npz')
import ast
#params = ast.literal_eval(str(trajdict['params']))
traj_closed_train = trajdict['traj_closed_train_hungarian']
traj_open_train = trajdict['traj_open_train_hungarian']
traj_closed_test = trajdict['traj_closed_test_hungarian']
traj_open_test = trajdict['traj_open_test_hungarian']
x = np.vstack([traj_closed_train, traj_open_train])
xval = np.vstack([traj_closed_test, traj_open_test])

In [19]:
batch_size =256
epochs = 10
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

In [20]:
train_set = traj_closed_train

In [21]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
data_path = Path(".").absolute().parent / "data"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [22]:
device = "cpu"